In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import operator

from util.api import get_pkgs_indirect_dependency, calculate_bottom, diagram_box

%matplotlib inline
plt.style.use('classic')

In [2]:
lts_list = ['0-7', '2-22', '3-22', '6-35', '7-24', '9-21', '11-22', '12-14', '12-26', '13-11', '13-19', '14-27', '15-3', '16-11']
df_list = []
df_by_category_list = []

for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../../data/lts-{lts}/lts-{lts}.df')
    df_list.append(df)
    
for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../../data/lts-{lts}/lts-{lts}-by-category.df')
    df_by_category_list.append(df)

In [ ]:
ind_dependence_package = get_pkgs_indirect_dependency(df_list)

In [ ]:
for lts in range (len(df_list)):
    print(lts_list[lts],'package: ',df_list[lts]['package'][df_list[lts]['len_ind_deps']==min(df_list[lts]['len_ind_deps'])][0],'total deps: ',min(df_list[lts]['len_ind_deps']))


In [ ]:
for lts in range (len(df_list)):
    len_deps = []
    for i,x in enumerate (list(df_list[lts]['deps'])):
        len_deps.append(len(x))
    df_list[lts]['len_deps'] = len_deps
    print(lts_list[lts],'package: ',df_list[lts]['package'][df_list[lts]['len_deps']==max(df_list[lts]['len_deps'])][0],'total deps: ',max(df_list[lts]['len_deps']))
    print(lts_list[lts],'package: ',df_list[lts]['package'][df_list[lts]['len_ind_deps']==max(df_list[lts]['len_ind_deps'])][0],'total deps: ',max(df_list[lts]['len_ind_deps']))
    

# Research Questions (RQs)

- **RQ 1.** What packages are the most used? Which of them are outside from the Stackage selection?
- **RQ 2.** How many dependencies of packages are outside from the Stackage selection (in)directly?
- **RQ 3.** How frequently are the packages updated? 

Regarding monads, we analyze the evolution of dependencies of monad modules available in the mtl package:

- **RQ 4.** How has the use of monads evolved?
- **RQ 5.** Are there packages that stopped using monads?

## **RQ 2.** How many packages are outside of Stackage due to his (in)direct dependencies?

Procesamos todos los snapshots que consideramos en nuestra investigacion. Realizamos una busqueda en profundidad a las dependencias de cada paquete de los LTS, buscando alguna dependencia (in)directa que no se encontrara en la seleccion. Si este era el caso, se agregaba a un conjunto de paquetes que luego contabilizamos por cada LTS. Finalmente, calculamos el porcentage que estos paquetes representaban en cada snapshot.

* Cambiar leyenda por: (in)directly

- Mas del 12% de los paquetes quedan fuera de la seleccion indireamente al usar paquetes que non se encuentran en stackage
- Inclusive este valor aumento con el tiempo llegando a alcanzar el 17% en (<año>) y manteniendose en un 15% durante todas las versiones.
- algunos de los paquetes que tienen mas impacto en este problea son: ['quickcheck', 'microlens', 'hunit', 'cabal']
-  en el caso de cabal, este paquete fue deprecado por uno nuevo llamado Cabal, sin embargo, existen paquetes que todavia lo utilizan


In [ ]:
count_dependency = {}
aver_direct_dependency = []
aver_indirect_dependency = []
data = []

for idx, df in enumerate(df_list):
    totaldir = 0
    totalind = 0
    for idx, row in df.iterrows():
        totaldir += row['len_deps']
        totalind += row['len_ind_deps']
    aver_direct_dependency.append(round(totaldir / len(df), 1))
    aver_indirect_dependency.append(round(totalind / len(df), 1))
    
count_values = list(count_dependency)
aver_values = list(aver_direct_dependency)
aver_ind_values = list(aver_indirect_dependency)

data.append(aver_values)
data.append(aver_ind_values)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.margins(0.015) 
plt.subplots_adjust(bottom=0.1, right=1.2, top=1)

colors = ['darkgrey','lightgrey']
dependences_type = ['Average directly dependences','Average indirectly dependences']
for idx, dependences in enumerate(dependences_type):
    bar = ax.bar(x=lts_list, height=data[idx], bottom=calculate_bottom(data, idx), width=0.8, color=colors[idx], align='center', label=dependences)
    ax.bar_label(bar, labels=list(data[idx]), label_type='center', color='black', fontsize=15)

ax.legend(loc='best')
ax.set_xlabel('LTS release',fontsize=18)
ax.set_ylabel('Number of dependences',fontsize=18)
ax.set_xticks(lts_list)
ax.set_xticklabels(lts_list,fontsize=15)
ax.set_yticks([0, 5, 10, 15, 20, 25, 30, 35])
ax.set_yticklabels(['0', '5', '10', '15', '20', '25', '30', '35'],fontsize=15)
plt.savefig("../../graphs/rq2.pdf", format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
aux_list =  []
for idx,df in enumerate (df_list):
    aux_list.append(df_list[idx]['len_deps'])
fig,ax = plt.subplots(figsize =(12, 8)) 
ax.boxplot(aux_list)
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14],lts_list,size = 'small', color = 'k')
ax.set_yticks([0,10,20, 30, 40, 50,60,70,80,90,100])
ax.set_yticklabels(['0','10','20', '30', '40', '50','60','70','80','90','100'],fontsize=15)
plt.show()

In [ ]:
aux_list =  []
for idx,df in enumerate (df_list):
    aux_list.append(df_list[idx]['len_ind_deps'])
fig,ax = plt.subplots(figsize =(12, 8)) 
ax.boxplot(aux_list)
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14],lts_list,size = 'small', color = 'k')
ax.set_yticks([0,20,40, 60, 80, 100,120,140,160,180,200])
ax.set_yticklabels(['0','20','40', '60', '80', '100','120','140','160','180','200'],fontsize=15)
plt.show()

In [ ]:
outlier_dir = diagram_box(list_direct_deps)
outlier_ind = diagram_box(list_indirect_deps)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax.margins(0.015) 
plt.subplots_adjust(right=1.1)

more_deps = ['Outliers', 'Limit of dependences','Q3','Q2','Q1']
colors = ['dimgrey', 'grey', 'darkgrey', 'silver', 'lightgrey', 'gainsboro']
for idx, more_deps in enumerate(more_deps):
    bar = ax.bar(x=lts_list, height=outlier_dir[idx], bottom=calculate_bottom(outlier_dir, idx), width=.8, color=colors[idx], align='center', label=more_deps)
    ax.bar_label(bar, labels=list(map(lambda per: f"{round(per)}%",outlier_dir[idx])), label_type='center', color='black',fontsize=15)


handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)

ax.set_xticks(lts_list)
ax.set_xticklabels(lts_list,fontsize=15)
ax.set_yticks([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
ax.set_yticklabels(['10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'],fontsize=15)
ax.set_xlabel('LTS release',fontsize=18)
ax.set_ylabel('Percentage of monad type use',fontsize=18)

plt.savefig("../../graphs/rq2_1.pdf", format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax.margins(0.015) 
plt.subplots_adjust(right=1.1)

more_deps = ['Outliers', 'Limit of dependences','Q3','Q2','Q1']
colors = ['dimgrey', 'grey', 'darkgrey', 'silver', 'lightgrey', 'gainsboro']
for idx, more_deps in enumerate(more_deps):
    bar = ax.bar(x=lts_list, height=outlier_ind[idx], bottom=calculate_bottom(outlier_ind, idx), width=.8, color=colors[idx], align='center', label=more_deps)
    ax.bar_label(bar, labels=list(map(lambda per: f"{round(per)}%",outlier_ind[idx])), label_type='center', color='black',fontsize=15)


handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)

ax.set_xticks(lts_list)
ax.set_xticklabels(lts_list,fontsize=15)
ax.set_yticks([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
ax.set_yticklabels(['10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'],fontsize=15)
ax.set_xlabel('LTS release',fontsize=18)
ax.set_ylabel('Percentage of monad type use',fontsize=18)

plt.savefig("../graphs/rq2_2.pdf", format='pdf', bbox_inches='tight')
plt.show()